In [1]:
from collections import defaultdict
import json

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [18]:
data = pd.read_csv("../DataSets/balanced_pop_unpop_features.csv", sep=",")
data['title'] = data['Unnamed: 0']
data = data.drop('Unnamed: 0', axis=1)
data.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,time_signature,valence,label,title
0,0.21400,0.666,178242,0.677,0.000000,2,0.0979,-5.743,0.0326,100.014,4,0.178,1,#Beautiful
1,0.01340,0.807,183750,0.916,0.000012,0,0.0787,-3.282,0.2260,127.973,4,0.651,1,#SELFIE
2,0.00162,0.791,279507,0.615,0.000065,6,0.0812,-6.149,0.0667,128.017,4,0.393,1,#thatPOWER
3,0.76300,0.707,275227,0.709,0.000000,11,0.2740,-3.979,0.3400,89.094,4,0.501,1,0 To 100 / The Catch Up
4,0.57000,0.629,250173,0.572,0.000000,5,0.1920,-7.733,0.0387,100.015,4,0.386,1,1-800-273-8255


Here we normalize all values to make the regression more easily understandable.

In [3]:
no_title = data.drop(['title', 'label'], axis=1)
z_scores=(no_title - no_title.mean())/no_title.std()
z_scores.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,time_signature,valence
0,0.234429,0.153280,-1.087287,-0.034082,-0.111864,-0.924928,-0.569818,0.078899,-0.665590,-0.776052,0.110072,-1.499069
1,-0.744047,1.169892,-0.953584,1.390222,-0.111643,-1.467484,-0.704061,1.114347,1.339700,0.212902,0.110072,0.635067
2,-0.801507,1.054532,1.370844,-0.403567,-0.110689,0.160186,-0.686582,-0.091923,-0.312020,0.214459,0.110072,-0.529007
3,2.912313,0.448891,1.266950,0.156620,-0.111864,1.516578,0.661443,0.821089,2.521722,-1.162309,0.110072,-0.041720
4,1.970908,-0.113491,0.658783,-0.659823,-0.111864,-0.111093,0.088113,-0.758380,-0.602341,-0.776016,0.110072,-0.560590


In [56]:
corr = no_title.corr()
total_corr = []
c = 0
for effect in corr:
    ind = 0
    for i in corr[effect]:
        if c == 0:
            total_corr.append(0)
        total_corr[ind] += abs(i)
        ind+=1
    c += 1
print(total_corr)
        


[2.7329680037073145, 2.1948466714079466, 1.6361565821815707, 3.560806103766256, 1.5099574136833491, 1.273709376402949, 1.786414884476453, 3.264330658962157, 1.9324017245762282, 1.884353906738323, 1.7827839781681039, 2.7386064388550784]


In [48]:
corr

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,time_signature,valence
acousticness,1.000000,-0.184256,-0.007125,-0.520882,0.069646,-0.003758,-0.066380,-0.419566,-0.036140,-0.146195,-0.104638,-0.174381
danceability,-0.184256,1.000000,-0.083887,-0.041506,0.008108,0.013918,-0.068204,0.057201,0.232517,-0.133679,0.097874,0.273697
duration_ms,-0.007125,-0.083887,1.000000,-0.089369,0.027760,-0.045954,0.056755,-0.090763,-0.018312,-0.004759,-0.025812,-0.185662
energy,-0.520882,-0.041506,-0.089369,1.000000,-0.054364,0.032311,0.171197,0.732734,-0.099753,0.193083,0.182769,0.442839
instrumentalness,0.069646,0.008108,0.027760,-0.054364,1.000000,0.033556,0.001823,-0.212182,-0.022305,0.001817,0.011647,-0.066750
key,-0.003758,0.013918,-0.045954,0.032311,0.033556,1.000000,0.012247,0.008563,0.078638,-0.002376,0.004328,0.038061
liveness,-0.066380,-0.068204,0.056755,0.171197,0.001823,0.012247,1.000000,0.074880,0.150872,0.068766,0.047602,0.067690
loudness,-0.419566,0.057201,-0.090763,0.732734,-0.212182,0.008563,0.074880,1.000000,-0.106264,0.127396,0.143307,0.291474
speechiness,-0.036140,0.232517,-0.018312,-0.099753,-0.022305,0.078638,0.150872,-0.106264,1.000000,0.113213,0.052869,0.021518
tempo,-0.146195,-0.133679,-0.004759,0.193083,0.001817,-0.002376,0.068766,0.127396,0.113213,1.000000,0.014236,0.078833


In [4]:
x_train, x_test, y_train, y_test= train_test_split(z_scores, data['label'])

Here we create a regression using every feature available in order to predict if a song will be popular or unpopular

In [5]:
clf1 = LogisticRegression()
clf1.fit(x_train, y_train)
predicted_train = clf1.predict(x_train)
predicted_test = clf1.predict(x_test)
print (clf1.coef_, clf1.intercept_)

[[ 0.00297953  0.02364152  0.09443518 -0.02099304 -0.08425741  0.060467
  -0.09656124  0.03852642  0.12609256 -0.01052694  0.08745966  0.08939907]] [-0.01425616]


An initial test of the R^2 values to determine predictive power. We see that the regression was able to explain 54% of the variability around the mean for the training data and 44.7% of the variability for the test data.

In [6]:
print(clf1.score(x_train, y_train))
print(clf1.score(x_test, y_test))

0.5485074626865671
0.4626865671641791


Now, we re-run the regression using the three variables with the highest coefficients in the original model, danceability, instrumentalness, and tempo.

In [7]:
test_2 = z_scores[['danceability', 'instrumentalness', 'tempo']]
x_train2, x_test2, y_train2, y_test2= train_test_split(test_2, data['label'])

In [8]:
clf2 = LogisticRegression()
clf2.fit(x_train2, y_train2)
predicted_train2 = clf2.predict(x_train2)
predicted_test2 = clf2.predict(x_test2)
print (clf2.coef_, clf2.intercept_)

[[ 0.11341328 -0.17543239  0.08884292]] [-0.01935708]


Here we see that limiting the number of variables we consider slightly increases our explaination power for the test data to 48.8%

In [9]:
print(clf2.score(x_train2, y_train2))
print(clf2.score(x_test2, y_test2))

0.527363184079602
0.5186567164179104


In [58]:
test_3 = z_scores[['key', 'instrumentalness', 'duration_ms']]
x_train3, x_test3, y_train3, y_test3= train_test_split(test_3, data['label'])

In [59]:
clf3 = LogisticRegression()
clf3.fit(x_train3, y_train3)
predicted_train3 = clf3.predict(x_train3)
predicted_test3 = clf3.predict(x_test3)
print (clf3.coef_, clf3.intercept_)

[[ 0.0264509  -0.00142037 -0.03638956]] [-0.02991092]


In [60]:
print(clf3.score(x_train3, y_train3))
print(clf3.score(x_test3, y_test3))

0.5
0.47761194029850745


In [65]:
test_4 = z_scores['instrumentalness']
test_4 = test_4.reshape(-1, 1)
x_train4, x_test4, y_train4, y_test4= train_test_split(test_4, data['label'])

/Applications/Anaconda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


In [66]:
clf4 = LogisticRegression()
clf4.fit(x_train4, y_train4)
predicted_train4 = clf4.predict(x_train4)
predicted_test4 = clf4.predict(x_test4)
print (clf4.coef_, clf4.intercept_)

[[-0.12723903]] [-0.03460261]


In [68]:
print(clf4.score(x_train4, y_train4))
print(clf4.score(x_test4, y_test4))

0.5074626865671642
0.47761194029850745
